In [128]:
import os
import sys
import logging
FORMAT = "%(filename)s:%(lineno)d %(funcName)s %(levelname)s %(message)s"
logging.basicConfig(format=FORMAT, level=logging.INFO)

import re
import imp
import itertools
import random
import numpy as np
from numpy import array
import pandas as pd
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
from matplotlib import pyplot as plt
#import seaborn as sns


# do this before we have a chance to import openbabel!
import rdkit, rdkit.Chem.rdDistGeom, rdkit.DistanceGeometry

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import rdBase

import py3Dmol

from rmgpy.molecule import Molecule
from rmgpy.species import Species
from rmgpy.reaction import Reaction


from multi_molecule import *
from multi_reaction import *

from ase.calculators.morse import * #chosing this calculator for now because it's fast
from ase.calculators.dftb import *
from ase.calculators.lj import *
from ase.calculators.emt import *
from ase.calculators.gaussian import *
from simple_es import *
from utilities import *
from brute_force import *

from copy import deepcopy

In [22]:
mol =Multi_Molecule("CCCCC")
mol.ase_molecule.set_calculator(EMT())
mol.view_mol()


In [23]:
initial_pop = create_initial_population(mol)

In [24]:
top_pop = perform_simple_es(mol, initial_pop)
top_pop

,Energy,Torsion 0,Torsion 1
44,5.789288,-39.712225,306.060632
37,5.789517,-39.900213,305.906406
131,5.789706,-42.302473,305.967841
185,5.789849,-37.883886,305.800722
79,5.789853,-40.740465,305.726766
128,5.790073,-40.144514,305.552623
157,5.790122,-39.699356,305.505777
177,5.790214,-37.889653,305.525048
76,5.790243,-38.572671,305.439441
83,5.790269,-38.533920,305.424357


In [25]:
top_pop.iloc[0,1:]

Torsion 0    -39.712225
Torsion 1    306.060632
Name: 44, dtype: float64

In [26]:
tor = mol.torsions[0]
tor.left_mask

[True,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True]

In [29]:
for i, dihedral in enumerate(top_pop.iloc[0,1:]):
    tor = mol.torsions[i]
    mol.ase_molecule.set_dihedral(tor.indices, angle=dihedral, mask=tor.right_mask)
mol.update_geometry_from_ase_mol()
mol.view_mol()

In [35]:
mol.ase_molecule.set_calculator(calc = Gaussian())# label="CCCC", scratch="/gss_gpfs_scratch/harms.n/", method="m062x", basis="6-311+g(2df,2p)"))


In [36]:
from ase.optimize import BFGS
mol.ase_molecule.set_calculator(calc = Gaussian())
opt = BFGS(mol.ase_molecule)
opt.run()

      Step     Time          Energy         fmax
BFGS:    0 12:12:53    -5341.520411        1.3230
BFGS:    1 12:12:58    -5341.616854        0.9794
BFGS:    2 12:13:04    -5341.759697        0.5725
BFGS:    3 12:13:10    -5341.837508        0.4847
BFGS:    4 12:13:15    -5341.886380        0.4798
BFGS:    5 12:13:21    -5341.931066        0.4861
BFGS:    6 12:13:26    -5341.963187        0.4902
BFGS:    7 12:13:32    -5341.981350        0.4508
BFGS:    8 12:13:37    -5341.998121        0.3779
BFGS:    9 12:13:43    -5342.020271        0.3869
BFGS:   10 12:13:48    -5342.042527        0.3829
BFGS:   11 12:13:54    -5342.063145        0.3374
BFGS:   12 12:13:59    -5342.079945        0.3352
BFGS:   13 12:14:05    -5342.092237        0.2934
BFGS:   14 12:14:10    -5342.105867        0.2550
BFGS:   15 12:14:16    -5342.117897        0.2062
BFGS:   16 12:14:21    -5342.124624        0.1881
BFGS:   17 12:14:27    -5342.129927        0.1868
BFGS:   18 12:14:32    -5342.135198        0.1804
B

True

In [37]:
mol.update_geometry_from_ase_mol()
mol.view_mol()

In [41]:
mol.torsions[1].dihedral

294.2613057335393

In [44]:
mol.ase_molecule.atoms

AttributeError: 'Atoms' object has no attribute 'atoms'

In [93]:
smiles_dict = {
    "Phe" : "NC(CC1=CC=CC=C1)C(=O)O",
    "Gly" : "NCCC(=O)O",
    "Ala" : "NC(C)C(=O)O",
    "Ile" : "NC(C(C)CC)C(=O)O",
    "Leu" : "NC(CC(C)C)C(=O)O",
    "Val" : "NC(C(C)C)C(=O)O",
    "Trp" : "NC(CC1=CNC2=C1(C=CC=C2))C(=O)O"   
}

In [96]:
i = 20 
a = list(smiles_dict.itervalues())[i % 7]
moll = Multi_Molecule(a)

In [97]:
moll.view_mol()

In [98]:
job_number = 1
i = job_number - 1

smiles = list(smiles_dict.itervalues())[i % 7]
mol =Multi_Molecule(smiles)
mol.ase_molecule.set_calculator(EMT())
initial_pop = create_initial_population(mol)
top_pop = perform_simple_es(mol, initial_pop)

for i, dihedral in enumerate(top_pop.iloc[0,1:]):
    tor = mol.torsions[i]
    mol.ase_molecule.set_dihedral(tor.indices, angle=dihedral, mask=tor.right_mask)
mol.update_geometry_from_ase_mol()
mol.view_mol()

In [99]:
for i, dihedral in enumerate(top_pop.iloc[0,1:]):
    tor = mol.torsions[i]
    mol.ase_molecule.set_dihedral(tor.indices, angle=dihedral, mask=tor.left_mask)
mol.update_geometry_from_ase_mol()
mol.view_mol()

In [100]:
from ase.optimize import BFGS
mol.ase_molecule.set_calculator(calc = Gaussian())
opt = BFGS(mol.ase_molecule)
opt.run()

      Step     Time          Energy         fmax
BFGS:    0 14:05:30   -11941.424624       10.0941
BFGS:    1 14:06:23   -11942.321590        6.3034
BFGS:    2 14:07:16   -11942.841844        4.2449
BFGS:    3 14:08:03   -11943.094880        1.9993
BFGS:    4 14:08:50   -11943.226012        1.0634
BFGS:    5 14:09:36   -11943.292440        1.1330
BFGS:    6 14:10:20   -11943.346240        1.1858
BFGS:    7 14:11:04   -11943.403501        1.1581
BFGS:    8 14:11:56   -11943.450908        0.8006
BFGS:    9 14:12:42   -11943.483108        0.4344
BFGS:   10 14:13:30   -11943.503026        0.4649
BFGS:   11 14:14:22   -11943.526515        0.5103
BFGS:   12 14:15:08   -11943.549879        0.6051
BFGS:   13 14:15:52   -11943.574255        0.5733
BFGS:   14 14:16:36   -11943.598514        0.5566
BFGS:   15 14:17:19   -11943.622827        0.5412
BFGS:   16 14:18:07   -11943.646801        0.5380
BFGS:   17 14:19:00   -11943.670825        0.5537
BFGS:   18 14:19:42   -11943.695082        0.5922
B

True

In [101]:
mol.update_geometry_from_ase_mol()
mol.view_mol()

In [127]:
mol.ase_molecule.write("test.xyz")

In [123]:
mol.ase_molecule.get_atomic_numbers()

array([6, 6, 6, 6, 6, 6, 7, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1])

In [125]:
a = ase.io.read("test.xyz")

Atoms(symbols='C6NOH10OH3', pbc=False, calculator=SinglePointCalculator(...))